# GtR organisations and sectors

Here we take a list of GtR organisations matched with Companies House and extract their SIC codes and sizes.

We will extract the SIC codes using a 4-digit sic code - Nesta segment lookup

# Preamble

In [ ]:
%run notebook_preamble.ipy

# Load data

In [ ]:
# This is the GtR-CH match

gtr_ch_matched = pd.read_csv('../data/external/gtr_house_metadata.csv',dtype={'company_number':str})

In [ ]:
gtr_ch_matched.columns

In [ ]:
# This is the 4-digit sic - industry segment lookup

industry_lookup = pd.read_csv('../data/external/industry_cluster_lookup_feb_2017.csv',
                             dtype={'sic_4':str})

In [ ]:
# And this is the Companies House data

ch = pd.read_csv('../data/external/ch_data/BasicCompanyDataAsOneFile-2019-07-01.csv',dtype={'CompanyNumber':str})


In [ ]:
ch.columns = [x.lower().strip() for x in ch.columns]

In [ ]:
ch = ch[['companyname','companynumber','companystatus','regaddress.postcode','siccode.sictext_1']]

## Merge

In [ ]:
gtr_ch_merged = pd.merge(gtr_ch_matched,ch,left_on='company_number',right_on='companynumber')

Some missing companies

In [ ]:
missing_numbers = set(gtr_ch_matched['company_number'])-set(ch['companynumber'])

In [ ]:
len(missing_numbers)

Check with Alex what these could be

In [ ]:
#Exterct four digit sic codes from the five digits

gtr_ch_merged['sic'] = [x.split(' ')[0] for x in gtr_ch_merged['siccode.sictext_1']]

#Deal with the presence of SIC codes with different lengths
gtr_ch_merged['sic_4'] = [x if len(x)==4 else x[:-1] if len(x)==5 else x+'0' for x in gtr_ch_merged['sic']] 

In [ ]:
merged_w_segments = pd.merge(gtr_ch_merged,industry_lookup,left_on='sic_4',right_on='sic_4')

In [ ]:
merged_w_segments.head()

In [ ]:
#There are a few sic codes missing from our lookup - unclear why
missing_sics = set(gtr_ch_merged['sic_4'])-set(industry_lookup['sic_4'])

## Read the link table matching projects to organisations

In [ ]:
link = pd.read_csv('../data/external/gtr_link_table.csv')

In [ ]:
link.columns

In [ ]:
project_org_match = pd.merge(merged_w_segments,link,left_on='id',right_on='id')

In [ ]:
# Finally - turn this into a project - org lookup

In [ ]:
project_sectors = pd.concat([project_org_match.groupby('project_id')[var].apply(lambda x: list(x)) for var in 
                             ['companyname','cluster','sic_4','descr']],axis=1)

In [ ]:
project_sectors.to_csv(f'../data/processed/{today_str}_gtr_organisations_industries_2.csv',compression='zip')

In [ ]:
project_sectors

In [ ]:
comp_names = [x[0].lower() for x in project_sectors['companyname']]

has_bbc = [x for x in comp_names if any(val in x for val in ['bbc','broadcasting'])]

In [ ]:
has_bbc